# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	8272 candidates
Dev set:	888 candidates
Test set:	4620 candidates


In [3]:
import numpy as np

# Load dev labels and convert to [0, 1] range
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
dev_labels = (np.ravel(L_gold_dev.todense()) + 1) / 2

# Computing features

For `SparseLogReg`

In [4]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

%time F_train = featurizer.apply(split=0)
F_train

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 8min 29s, sys: 4.02 s, total: 8min 33s
Wall time: 8min 37s


<8272x122840 sparse matrix of type '<type 'numpy.int64'>'
	with 448906 stored elements in Compressed Sparse Row format>

In [5]:
%%time
F_dev  = featurizer.apply_existing(split=1)
F_test = featurizer.apply_existing(split=2)

Clearing existing...
Running UDF...
[========================================] 100%

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 4min 50s, sys: 1.27 s, total: 4min 52s
Wall time: 4min 52s


# Training `SparseLogReg`

Instead of LSTM, to start.  First, reloading training marginals:

In [6]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [13]:
from snorkel.learning import SparseLogisticRegression
from snorkel.learning.utils import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

# NOTE: A larger search (n) would likely lead to a higher score!
searcher = RandomSearch(SparseLogisticRegression, [rate_param, l1_param, l2_param], F_train,
                        Y_train=train_marginals, n=5)

Initialized RandomSearch search of size 5. Search space size = 125.


In [14]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

<888x1 sparse matrix of type '<type 'numpy.int64'>'
	with 888 stored elements in Compressed Sparse Row format>

In [23]:
%%time
import numpy as np
np.random.seed(1701)
disc_model, run_stats = searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=0.5, print_freq=25)

[1] Testing lr = 1.00e-02, l1_penalty = 1.00e-06, l2_penalty = 1.00e-04
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=6736  #epochs=50  batch size=256
[SparseLogisticRegression] Epoch 0 (0.38s)	Average loss=0.780147
[SparseLogisticRegression] Epoch 25 (12.20s)	Average loss=0.711312
[SparseLogisticRegression] Epoch 49 (23.06s)	Average loss=0.717617
[SparseLogisticRegression] Training done (23.06s)
[SparseLogisticRegression] F1 Score: 0.496679946879
[SparseLogisticRegression] Model saved as <SparseLogisticRegression_0>
[2] Testing lr = 1.00e-04, l1_penalty = 1.00e-06, l2_penalty = 1.00e-03
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=6736  #epochs=50  batch size=256
[SparseLogisticRegression] Epoch 0 (0.38s)	Average loss=1.412612
[SparseLogisticRegression] Epoch 25 (11.81s)	Average loss=1.277493
[SparseLogisticRegression] Epoch 49 (23.05s)	Average loss=1.242918
[SparseLogisticRegression] Training done (23.05s)
[SparseLogistic

In [24]:
run_stats

,lr,l1_penalty,l2_penalty,Prec.,Rec.,F1
2,0.0010,0.010000,0.0001,0.415170,0.702703,0.521957
1,0.0001,0.000001,0.0010,0.417031,0.645270,0.506631
3,0.0010,0.000100,0.0010,0.407098,0.658784,0.503226
0,0.0100,0.000001,0.0001,0.409190,0.631757,0.496680
4,0.0100,0.000001,0.0001,0.404301,0.635135,0.494087


### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

In [16]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')

AnnotatorLabels created: 0


In [17]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

<4620x1 sparse matrix of type '<type 'numpy.int64'>'
	with 4620 stored elements in Compressed Sparse Row format>

In [18]:
_, _, _, _ = disc_model.error_analysis(session, F_test, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.738
Neg. class accuracy: 0.523
Precision            0.428
Recall               0.738
F1                   0.541
----------------------------------------
TP: 1110 | FP: 1485 | TN: 1630 | FN: 395



# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

**Note: Again, training for more epochs than below will greatly improve performance- try it out!**

In [20]:
from snorkel.learning import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   10,
    'dropout':    0.5,
    'rebalance':  0.25,
    'print_freq': 5
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train, train_marginals, X_dev=dev, Y_dev=dev_labels, **train_kwargs)

[reRNN] Training model
[reRNN] n_train=4491  #epochs=10  batch size=256
[reRNN] Epoch 0 (22.25s)	Average loss=0.664210	Dev F1=19.63
[reRNN] Epoch 5 (130.85s)	Average loss=0.591362	Dev F1=45.76
[reRNN] Epoch 9 (216.73s)	Average loss=0.576549	Dev F1=47.54
[reRNN] Model saved as <reRNN>
[reRNN] Training done (219.80s)
[reRNN] Loaded model <reRNN>
